In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import graph_creation
import time
import holidays
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
df = pd.read_excel('./GEFCom2014-E.xlsx')
df['dow'] = df.Date.apply(lambda x: x.dayofweek)
df['doy'] = df.Date.apply(lambda x: x.dayofyear)
df['month'] = df.Date.apply(lambda x: x.month)
df = df[df.load.isnull().sum():]
df = df.reindex(columns=('doy', 'month', 'dow', 'Hour', 'T', 'load', 'Date'))
offset = df.load.mean()
scale = df.load.std()
df.load -= df.load.mean()
df.load /= df.load.std()
df['T'] -= df['T'].mean()
df['T'] /= df['T'].std()

ush = holidays.US()
df['is_holiday'] = df.Date.apply(lambda x: x in ush)

df.month = np.cos(2*np.pi/12*df.month)
df.Hour = np.cos(2*np.pi/24*df.Hour)
df.dow = np.cos(2*np.pi/7*df.dow)

df[['daily_load', 'daily_T']] = df[['load', 'T']].rolling(24).mean()
df = df[23:].reset_index(drop=True)
del df['Date'], df['doy']

In [4]:
df.head()

,month,dow,Hour,T,load,is_holiday,daily_load,daily_T
0,0.866025,0.62349,1.000000,-1.210365,-0.557087,True,0.111100,-1.360818
1,0.866025,1.00000,0.965926,-1.210365,-0.905481,True,0.094786,-1.357201
2,0.866025,1.00000,0.866025,-1.227725,-1.077954,True,0.082570,-1.349968
3,0.866025,1.00000,0.707107,-1.227725,-1.153842,True,0.074018,-1.344181
4,0.866025,1.00000,0.500000,-1.227725,-1.153842,True,0.069275,-1.333331


In [5]:
train = df[:7*len(df)//8]
# val = df[3*len(df)//4:7*len(df)//8]
test = df[7*len(df)//8:]

In [6]:
def s2s_sequencify_withdaily(df, n_past: int=24, n_pred: int=24, days_past: int=21):
    x_encoder = []  # encoder inputs
    x_decoder = []  # decoder inputs
    daily_encoder = []
    targets = []
    offset = max(n_past, days_past*24)+1
    daily_columns = ['month', 'dow', 'Hour', 'is_holiday'] + [col for col in list(df.columns) if 'daily' in col]
    daily_df = df[daily_columns]
    daily_array = np.array(daily_df)
    hourly_columns = ['Hour','month','dow','is_holiday','T','load']
    hourly_df = df[hourly_columns]
    x = np.array(hourly_df)
    y = x[:,-1]
    x = x[:,:-1]
    for i in np.arange(offset, len(y) - n_pred):
        x_encoder.append(np.append(x[i - n_past:i],
                            y[i - n_past:i].reshape(-1, 1), axis=1))
        x_decoder.append(x[i:i + n_pred])
        daily_encoder.append(daily_array[i-24*days_past-1:i-1+24:24])
        targets.append(y[i:i + n_pred])
    return np.array(x_encoder), np.array(x_decoder), np.array(daily_encoder), np.array(targets)
def batch(*vars, size=512):
    for i in range(0, min(len(v) for v in vars), size):
        yield (v[i:i+size] for v in vars)

In [7]:
xt, xtf, xtd, yt = s2s_sequencify_withdaily(train)
# xv, xvf, xvd, yv = s2s_sequencify_withdaily(val)
xtt, xttf, xttd, ytt = s2s_sequencify_withdaily(test)

In [8]:
hidden_dim = 32
daily_dim = 16

In [9]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder(tf.float32, (None, None, xt.shape[2]), name='x_past')
    xd = tf.placeholder(tf.float32, (None, None, xt.shape[2]), name='x_daily')
    y = tf.placeholder(tf.float32, (None, None), name='y')
    xf = tf.placeholder(tf.float32, (None, None, xtf.shape[2]), name='x_future')
    
    keep_prob = tf.placeholder_with_default(1.0, (), name='keep_prob')
    is_training = tf.placeholder_with_default(False, (), name='is_training')
    regularization = tf.placeholder_with_default(0.005, (), name='regularization')
    
    out_weight = tf.Variable(tf.random_normal((hidden_dim,))/hidden_dim, dtype=tf.float32, name='out_weight')
    out_bias = tf.Variable(tf.zeros(1), dtype=tf.float32, name='out_bias')
    
    # learnable affine transformation
    outputs  = graph_creation.other_s2s_lstm_multiresolution(x, xf, xd, hidden_dim, 2, daily_dim, use_bn=True, is_training=is_training, keep_prob=keep_prob)
    
    preds = tf.add(tf.einsum('ijk,k->ij', outputs, out_weight), out_bias, name='predictions')
    
    loss = tf.reduce_mean((y-preds)**2)
    reg_loss = tf.nn.l2_loss(out_weight) * regularization / tf.cast(tf.shape(x)[0], tf.float32)
    step = tf.train.AdamOptimizer().minimize(loss)
    saver = tf.train.Saver()

In [10]:
try:
    while True:
        tf.get_default_session().close()
except:
    pass
sess = tf.InteractiveSession(graph=g)
sess.run(tf.global_variables_initializer())

In [11]:
def train_epoch(bs, keep_p=1.0):
    perm = np.random.permutation(len(xt))
    errors = []
    for xs, xfs, xds, ys in batch(xt[perm], xtf[perm], xtd[perm], yt[perm], size=bs):
        _, l= sess.run((step, loss), feed_dict={x:xs, xf:xfs, xd:xds, y:ys, keep_prob:keep_p, is_training:True})
        errors.append(l)
    return errors

def evaluate(bs):
    l = []
    s = 0
    for xs, xfs, xds, ys in batch(xtt, xttf, xttd, ytt, size=bs):
        l.append(sess.run(loss, feed_dict={x:xs, xf:xfs, xd:xds, y:ys})*len(ys))
        s += len(ys)
    return sum(l)/s

In [12]:
bs = 1024

In [13]:
evaluate(bs*2) # run to make sure everything works

0.9065477612573106

In [14]:
train_errs = []
test_errs = []
for i in range(len(test_errs), 300):
    start = time.time()
    train_err = train_epoch(bs, 0.7)
    train_errs.extend(train_err)
    test_errs.append(evaluate(bs*2))
    end = time.time()

    if i>1 and test_errs[-1] == min(test_errs):
        saver.save(sess, './models/multires/model', global_step=len(test_errs))
    print(f'Epoch {i} ({end-start:.2f}s): train_loss={train_errs[-1]:.4f}, test_loss={test_errs[-1]:.4f}')

Epoch 0 (25.79s): train_loss=0.1362, test_loss=0.1644
Epoch 1 (24.96s): train_loss=0.0961, test_loss=0.0585
Epoch 2 (24.81s): train_loss=0.0744, test_loss=0.0480
Epoch 3 (25.79s): train_loss=0.0661, test_loss=0.0415
Epoch 4 (25.52s): train_loss=0.0610, test_loss=0.0362
Epoch 5 (25.65s): train_loss=0.0572, test_loss=0.0349
Epoch 6 (25.03s): train_loss=0.0550, test_loss=0.0346
Epoch 7 (25.75s): train_loss=0.0542, test_loss=0.0311
Epoch 8 (24.50s): train_loss=0.0504, test_loss=0.0305
Epoch 9 (24.37s): train_loss=0.0472, test_loss=0.0307
Epoch 10 (24.24s): train_loss=0.0472, test_loss=0.0264
Epoch 11 (24.15s): train_loss=0.0470, test_loss=0.0260
Epoch 12 (24.41s): train_loss=0.0471, test_loss=0.0269
Epoch 13 (24.03s): train_loss=0.0436, test_loss=0.0277
Epoch 14 (24.26s): train_loss=0.0429, test_loss=0.0239
Epoch 15 (23.94s): train_loss=0.0411, test_loss=0.0244
Epoch 16 (24.26s): train_loss=0.0437, test_loss=0.0250
Epoch 17 (24.48s): train_loss=0.0411, test_loss=0.0248
Epoch 18 (23.89s): t

Epoch 149 (25.21s): train_loss=0.0279, test_loss=0.0164
Epoch 150 (23.00s): train_loss=0.0287, test_loss=0.0161
Epoch 151 (21.87s): train_loss=0.0273, test_loss=0.0168
Epoch 152 (21.78s): train_loss=0.0268, test_loss=0.0156
Epoch 153 (24.69s): train_loss=0.0278, test_loss=0.0158
Epoch 154 (24.18s): train_loss=0.0268, test_loss=0.0158
Epoch 155 (23.45s): train_loss=0.0273, test_loss=0.0159
Epoch 156 (22.93s): train_loss=0.0276, test_loss=0.0167
Epoch 157 (22.26s): train_loss=0.0264, test_loss=0.0161
Epoch 158 (21.64s): train_loss=0.0274, test_loss=0.0163
Epoch 159 (21.85s): train_loss=0.0277, test_loss=0.0157
Epoch 160 (23.81s): train_loss=0.0289, test_loss=0.0167
Epoch 161 (24.08s): train_loss=0.0277, test_loss=0.0147
Epoch 162 (21.69s): train_loss=0.0263, test_loss=0.0152
Epoch 163 (23.43s): train_loss=0.0272, test_loss=0.0160
Epoch 164 (21.55s): train_loss=0.0278, test_loss=0.0152
Epoch 165 (21.83s): train_loss=0.0268, test_loss=0.0166
Epoch 166 (22.76s): train_loss=0.0278, test_loss

KeyboardInterrupt: 

In [ ]:
def predict(bs=bs, kp=1.0):
    ps = []
    for xs, xfs, xds, ys in batch(xtt, xttf, xttd, ytt, size=bs):
        ps.append(sess.run(preds, feed_dict={x:xs, xf:xfs, xd:xds, keep_prob:kp}))
    return np.concatenate(ps,0)

In [ ]:
pos = predict(bs*2)*scale + offset
yov = ytt*scale + offset
loss_by_horizon = ((yov-pos)**2).mean(0).astype(np.float32)
loss_by_horizon

In [ ]:
plt.plot(np.sqrt(loss_by_horizon))
plt.xlabel('Prediction Horizon')
plt.ylabel('RMSE')

In [ ]:
plt.plot((np.abs(yov-pos)/yov).mean(0).astype(np.float32) * 100)
plt.xlabel('Prediction Horizon')
plt.ylabel(r'MAPE (%)')